# Uninformed Search

This is the most basic form of seach.It implies trying to reach a goal state with no knowledge of where such goal state is.

Threfore, every possible strategy will just move from one node to another and check whether we reached the goal state or not. It will keep expanding until we reach the goal state.

We have an intrinsic measure of distance (or better, a *cost*) within a graph of nodes and paths, even if the absence of knowledge regarding the gold state: the number of steps associated to a given path. We can also consider adding other measure of distance given for example by the number of kilometers between the start state and our current state.

Given a graph with a cost defined for the paths, we can imagine 3 ways of searching.

- **Breadth-First Search (BFS)**: We start out with the root node and then expand the so-called frotier of examined nodes by considering other nodes that have the same *depth*. Here, the same depth means that the distance between the root node and each of the next-to-consider nodes is the same. In terms of the picture of a tree, this implies considering first expansions of the same depth. When there are ties (i.e. two nodes are one step away from current node) we can just pick one at random or use some rule of thumb.

- **Depth-First Search (DFS)**: We expand the *deepest* node in the current frontier of the tree. This means that the search proceeds to the deepest level of the current node, up to the point where nodes have no successors. As those nodes are exanded, they are removed from the frontier, so then the algorithm backs up to the next deepest node. 


![BFSvsDFS](./DFSBFS.png)

The implementation of Uninformed Search in pseudocode is a follows:


```function Graph.Search(problem):
  frontier = {[initial]}; explored={}
  loop:
    if frontier is empty: return FAIL
    path = remove.choice(frontier)
    s = path.end; add s to explored
    if s is a goal: return path
    for a in actions:
        add[path + a -> Result(s,a)]
        to frontier
        unless Result(s,a) in frontier or explored
```

So we first have a frontier initialized with with the root node as the only node. Then, we remove a path from the frontier and consider the state `s` that the removed path leads to. Then we add `s` to the explored list. If `s` is the goal state, we return the path. If not, we add to the frontier all the paths that can be constructed starting from `s` and adding each and all of the possible actions `a` (we do this unless `s+a` is in the explored list or is already in the frontier).

The difference between BFS and DFS is simply the path that is removed from the frontier at each iteration, i.e. the particular function or routine that we apply the line `path = remove.choice(frontier)` above. 


## Graph

In [4]:
class GraphNode(object):
    def __init__(self, val):
        self.value = val
        self.children = []
        
    def add_child(self, new_node):
        self.children.append(new_node)
    
    def remove_child(self, del_node):
        if del_node in self.children:
            self.children.remove(del_node)

class Graph(object):
    def __init__(self, node_list):
        self.nodes = node_list
        
    def add_edge(self, node1, node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.add_child(node2)
            node2.add_child(node1)
            
    def remove_edge(self, node1, node2):
        if(node1 in self.nodes and node2 in self.nodes):
            node1.remove_child(node2)
            node2.remove_child(node1)

In [224]:
# create a graph
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS, nodeH, nodeG, nodeP, nodeR, nodeA]) 
graph1.add_edge(nodeG, nodeR)
graph1.add_edge(nodeA, nodeR)
graph1.add_edge(nodeA, nodeG)
graph1.add_edge(nodeR, nodeP)
graph1.add_edge(nodeH, nodeG)
graph1.add_edge(nodeH, nodeP)
graph1.add_edge(nodeS, nodeR)

In [6]:
# To verify that the graph is created accurately.
# Let's just print all the parent nodes and child nodes.
for each in graph1.nodes:
    print('parent node = ',each.value,end='\nchildren\n')
    for each in each.children:
        print(each.value,end=' ')
    print('\n')

parent node =  S
children
R 

parent node =  H
children
G P 

parent node =  G
children
R A H 

parent node =  P
children
R H 

parent node =  R
children
G A P S 

parent node =  A
children
R G 



## Breadth-First Search

For this algorithm, the frontier is better to be represented with a FIFO (first-in-first-out) queue, which in python can be a usual queue or even a list

In [16]:
from queue import Queue
namequeue = Queue()
# Add elements
namequeue.put("Alice")
namequeue.put("Bob")
namequeue.put("Charlie")

# Remove elements
print(namequeue.get())
print(namequeue.get())
print(namequeue.get())

Alice
Bob
Charlie


In [4]:
namelist = []
namelist.append("Charlie")
namelist.append("Bob")
namelist.append("Alice")

# Remove elements
print(namelist.pop())
print(namelist.pop())
print(namelist.pop())

Alice
Bob
Charlie


Now we moving through the above graph using BFT. Implement the `bfs_search` to return the `GraphNode` with the value `search_value` starting at the `root_node`.

In [18]:
def bfs_search(root_node, search_value):
    bfs_frontier = Queue()
    bfs_frontier.put(root_node)
    bfs_explored = {}

    while True:
        if bfs_frontier.empty():
            return False

        # get (shallowest) node from frontier
        # ensured to be shallowest as frontier is FIFO
        current_node = bfs_frontier.get()
        # note: we don't check gold state here!
        bfs_explored.add(current_node)

        # consider the result of each possible action on 
        # the current node
        for next_node in current_node.children:
            # bfs_frontier is not iterable but bfs_frontier.queue is
            if (
                (next_node not in bfs_explored) and (next_node not in bfs_frontier.queue) 
                ):
                if next_node.value == search_value:
                    # return child node if condition is satisfied
                    # next_node is goal
                    return next_node
                # add child nodes to the frontier
                bfs_frontier.put(next_node)


In [19]:
assert nodeA == bfs_search(nodeS, 'A')
assert nodeS == bfs_search(nodeP, 'S')
assert nodeR == bfs_search(nodeH, 'R')

### Faster solution (Udacity)

In [ ]:
# Solution
def bfs_search(root_node, search_value):
    visited = set()                           # Sets are faster while lookup. Lists are faster to iterate.
    queue = [root_node]
    
    while len(queue) > 0:
        current_node = queue.pop(0)
        visited.add(current_node)

        if current_node.value == search_value:
            return current_node

        for child in current_node.children:
            if child not in visited:          # Lookup
                queue.append(child)


## Depth-First search

For this type of search a LIFO (last-in-first-out) queue is used to model the frontier, as the latest node to be incorporated is also the first one wose children will be considered.

In [1]:
import queue
lifonamequeue = queue.LifoQueue()
# Add elements
lifonamequeue.put("Alice")
lifonamequeue.put("Bob")
lifonamequeue.put("Charlie")

# Remove elements
print(lifonamequeue.get())
print(lifonamequeue.get())
print(lifonamequeue.get())


Charlie
Bob
Alice


We see that the last element to be put is the first one out. In this sense, it goes contrary to a LIFO queue.

In [241]:
def dfs_search_recursion(start_node, search_value, visited_nodes = set(), lifo_nodes=[]):
    # start_node works also as current node
    # finish if the find the goal
    if start_node.value == search_value:
        return start_node

    # add current node to visited nodes
    visited_nodes.add(start_node)
    # add all the non-visited children of the current node
    # (located at the end of the LIFO queue!)
    lifo_nodes.extend(list(set(start_node.children) - set(visited_nodes)))
    

    while len(lifo_nodes) > 0:
        # extract the last node from the LIFO queue
        # this will be one of the latest added nodes
        child = lifo_nodes.pop(-1)
        # process most-recently added children first: DFS
        if child not in visited_nodes: # redundant in principle: kept just in case
            return dfs_search_recursion(child, search_value, visited_nodes, lifo_nodes)

In [242]:
assert nodeA == dfs_search_recursion(nodeG, 'A', visited_nodes = set(), lifo_nodes=[])
assert nodeA == dfs_search_recursion(nodeS, 'A', visited_nodes = set(), lifo_nodes=[])
assert nodeS == dfs_search_recursion(nodeP, 'S', visited_nodes = set(), lifo_nodes=[])
assert nodeR == dfs_search_recursion(nodeH, 'R', visited_nodes = set(), lifo_nodes=[])

In [243]:
for i in range(100000):
    if i % 10000 == 0:
        print(i)
    if dfs_search_recursion(nodeG, 'A', visited_nodes = set(), lifo_nodes=[]).value  != 'A':
        print('errr')
        break

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


### Udacity's solution (WRONG!)

In [233]:
# Solution
def dfs_recursion_start(start_node, search_value):
    visited = set()               # Set to keep track of visited nodes.
    return dfs_recursion(start_node, visited, search_value)

# Recursive function
def dfs_recursion(node, visited, search_value):
    if node.value == search_value:
        found = True              # Don't search in other branches, if found = True
        return node
    
    visited.add(node)
    found = False
    result = None

    # Conditional recurse on each neighbour
    for child in node.children:
        if (child not in visited):
                result = dfs_recursion(child, visited, search_value)
                
                # Once the match is found, no more recurse 
                if found:
                    break
    return result

Hi,

I have noticed that if I remove the direct connection `G <-> A` in the graph defined in the exercise notebook of the DFS algorithm I get an assertion error in the following line:

`assert nodeA == dfs_recursion_start(nodeG, 'A')`

However, I would expect the DFS algorithm to still find the goal node through the path `G -> R -> A`, so there should be some issue with the proposed solution.

Assuming that I am not missing anything, I believe that the problem has to do with the fact that reassignment

`found = True`

has only the scope of the current node, and not on the previous node in the recursive function. Therefore, the break condition within the loop is not always triggered when it should.

I think that the current solution will crash whenever the goal node is a leaf node that is not directly connected with start_node, and the reason why it works for the graph currently defined in the exercise if because there is an interplay between visited nodes and connections so that the goal node happens to always be the last node to be checked.

Is there anything that I am not interpreting correctly?

Thanks

In [235]:
# create a graph
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS, nodeH, nodeG, nodeP, nodeR, nodeA]) 
graph1.add_edge(nodeG, nodeR)
graph1.add_edge(nodeA, nodeR)
#graph1.add_edge(nodeA, nodeG)
graph1.add_edge(nodeR, nodeP)
graph1.add_edge(nodeH, nodeG)
graph1.add_edge(nodeH, nodeP)
graph1.add_edge(nodeS, nodeR)

assert nodeA == dfs_recursion_start(nodeG, 'A')
assert nodeA == dfs_recursion_start(nodeS, 'A')
assert nodeS == dfs_recursion_start(nodeP, 'S')
assert nodeR == dfs_recursion_start(nodeH, 'R')

AssertionError: 

### Correcting Udacity's solution

Basically we can write a workaround for Udacity's solution while keeping as much as the original code as possible by simply ensuring that variable `found` is propagated as expected.

In [236]:
# Solution
def dfs_recursion_start(start_node, search_value):
    visited = set()               # Set to keep track of visited nodes.
    return dfs_recursion(start_node, visited, search_value)

# Recursive function
def dfs_recursion(node, visited, search_value):
    if node.value == search_value:
        return node, True
    visited.add(node)
    found = False
    result = None

    # Conditional recurse on each neighbour
    for child in node.children:
        if (child not in visited):
                result, found = dfs_recursion(child, visited, search_value)
                # Once the match is found, no more recurse 
                if found:
                    break
    return result, found

In [240]:
# create a graph
nodeG = GraphNode('G')
nodeR = GraphNode('R')
nodeA = GraphNode('A')
nodeP = GraphNode('P')
nodeH = GraphNode('H')
nodeS = GraphNode('S')

graph1 = Graph([nodeS, nodeH, nodeG, nodeP, nodeR, nodeA]) 
graph1.add_edge(nodeG, nodeR)
graph1.add_edge(nodeA, nodeR)
#graph1.add_edge(nodeA, nodeG)
graph1.add_edge(nodeR, nodeP)
graph1.add_edge(nodeH, nodeG)
graph1.add_edge(nodeH, nodeP)
graph1.add_edge(nodeS, nodeR)

assert nodeA == dfs_recursion_start(nodeG, 'A')[0]
assert nodeA == dfs_recursion_start(nodeS, 'A')[0]
assert nodeS == dfs_recursion_start(nodeP, 'S')[0]
assert nodeR == dfs_recursion_start(nodeH, 'R')[0]